In [ ]:
from huggingface_hub import ModelCard, ModelCardData
!pip install -qU transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import Dataset
from datasets import load_dataset
from trl import SFTTrainer
from jinja2 import Template
import yaml
import json
from pathlib import Path 

In [ ]:
pip install tf-keras

In [ ]:

# cwd = Path.cwd()
cwd = r'C:\Users\raush\Desktop\DeskTop\3.2 years\Junior AI\data' #TODO remove 
cwd = Path(cwd) 
datamodel = cwd / 'DataModel'
json_files = list(datamodel.glob('*/*.json'))
json_data = [json.load(i.open()) for i in json_files]

In [ ]:
# import json
# from pathlib import Path

# # Set the current working directory
# cwd = r'C:\Users\raush\Desktop\DeskTop\3.2 years\Junior AI\data'  # TODO: remove
# cwd = Path(cwd)

# # Define the path to the DataModel directory
# datamodel = cwd / 'DataModel'

# # Find all JSON files in the subdirectories of DataModel
# json_files = list(datamodel.glob('*/*.json'))

# # Load JSON data from each file
# json_data = [json.load(i.open()) for i in json_files]

# # Define the path for the output JSON file
# output_file = cwd / 'combined_data.json'

# # Save the combined JSON data to the output file
# with output_file.open('w') as f:
#     json.dump(json_data, f, indent=4)

# print(f"Combined JSON data has been saved to {output_file}")


In [ ]:
import pandas as pd
df = pd.read_csv(r"C:\Users\raush\Desktop\DeskTop\3.2 years\Junior AI\data\reference-table.csv")
df.loc[df['Column Name '] == 'Order Status', 'Column Name '] = 'order_status'
df.loc[df['Column Name '] == 'Shipping Method', 'Column Name '] = 'shipping_method'
df.to_csv('updated.csv', index=False)

In [ ]:
import pandas as pd

data = pd.read_csv(r"C:\Users\raush\Desktop\DeskTop\3.2 years\Junior AI\data\updated.csv")

json_data_ = data.to_json(orient="records")  

with open("updated_reference.json", "w") as f:
    f.write(json_data_)

In [ ]:

MODEL = "microsoft/Phi-3-mini-4k-instruct"
NEW_MODEL_NAME = "NewPHI"
DATASET_NAME = json_data 
SPLIT = "train"
MAX_SEQ_LENGTH = 2048

In [ ]:

model = AutoModelForCausalLM.from_pretrained(MODEL, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

dataset = load_dataset(DATASET_NAME, split=SPLIT)
EOS_TOKEN = tokenizer.eos_token_id

In [ ]:
# Create trainer and train model
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
)

trainer.train()

In [ ]:

data_model = json.load(open(r"C:\Users\raush\Desktop\DeskTop\3.2 years\Junior AI\data\combined_data.json"))
relationships = json.load(open(r"C:\Users\raush\Desktop\DeskTop\3.2 years\Junior AI\data\updated_reference.json"))
questions = open(r"C:\Users\raush\Desktop\DeskTop\3.2 years\Junior AI\data\Questions.csv").readlines()  

def extract_keywords(question):
    
    keywords = question.lower().split()  
    return keywords

def create_prompt(question):
    keywords = extract_keywords(question)

    tables = []
    columns = []
    for table in data_model["table_information"]:
        if any(keyword in table["table_name"].lower() or any(keyword in col["name"].lower() for col in table["columns"]) for keyword in keywords):
            tables.append(table)
            columns.extend(col for col in table["columns"] if any(keyword in col["name"].lower() for keyword in keywords))

    prompt = f"Example Question\n{question.strip()}\nExample Output:\nColumns\n"

    for table in tables:
        prompt += f"\n{{\n\"table_name\": \"{table['table_name']}\",\n\"table_description\": \"{table['table_description']}\",\n\"primary_key_column\": {table['primary_key_column']},\n\"columns\": [\n"
        for col in table["columns"]:
            prompt += f"{{\n\"name\": \"{col['name']}\",\n\"description\": \"{col['description']}\",\n\"data_type\": \"{col['data_type']}\",\n\"format\": \"{col['format']}\",\n\"is_pii_column\": \"{col['is_pii_column']}\",\n\"enum\": {col['enum']},\n\"dimension_group\": \"{col['dimension_group']}\",\n\"is_nullable\": \"{col['is_nullable']}\"\n}},\n"
        prompt = prompt.rstrip(",\n")  
        prompt += "\n]\n},\n"

  
    prompt += "\nRelationship\n"
    prompt += f"\"relationships\": {relationships}\n"

    return prompt

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(NEW_MODEL_NAME)

In [ ]:
# Generate a response
model = AutoModelForCausalLM.from_pretrained(model)

pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

for question in questions:
    prompt = create_prompt(question.strip())
    outputs = pipeline(prompt, max_length=50, num_return_sequences=1)
    print(outputs)